In [0]:
from pyspark.sql.functions import col, sum
from pyspark.sql.types import IntegerType, DoubleType, DateType, StringType

In [0]:
configs = {
  "fs.azure.account.auth.type": "CustomAccessToken",
  "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
}

directory_to_check = "/mnt/silver"
mounted_directories = [mount.mountPoint for mount in dbutils.fs.mounts()]

if directory_to_check in mounted_directories:
    print(f"The directory {directory_to_check} is already mounted.")
else:
    dbutils.fs.mount(
    source = "abfss://bronze@youtubeadlsg2.dfs.core.windows.net/",
    mount_point = "/mnt/silver",
    extra_configs = configs)

The directory /mnt/silver is already mounted.


In [0]:
df = spark.read.format('delta').load('/mnt/silver/data/')

In [0]:
df.show(5)

+----+--------------------+-----------+-----------+----------------+--------------------+-------+-------------+------------+-------------+----------------+------------+-----------------+--------------------------------+-----------------------+------------------------+----------------------+-----------------------+----------------------------+------------+-------------+------------+-----------------------------------+----------+-----------------+----------------+---------+----------+
|rank|            Youtuber|subscribers|video_views|        category|               Title|uploads|      Country|Abbreviation| channel_type|video_views_rank|country_rank|channel_type_rank|video_views_for_the_last_30_days|lowest_monthly_earnings|highest_monthly_earnings|lowest_yearly_earnings|highest_yearly_earnings|subscribers_for_last_30_days|created_year|created_month|created_date|Gross_tertiary_education_enrollment|Population|Unemployment_rate|Urban_population| Latitude| Longitude|
+----+------------------

In [0]:
df.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in df.columns)).show()


+----+--------+-----------+-----------+--------+-----+-------+-------+------------+------------+----------------+------------+-----------------+--------------------------------+-----------------------+------------------------+----------------------+-----------------------+----------------------------+------------+-------------+------------+-----------------------------------+----------+-----------------+----------------+--------+---------+
|rank|Youtuber|subscribers|video_views|category|Title|uploads|Country|Abbreviation|channel_type|video_views_rank|country_rank|channel_type_rank|video_views_for_the_last_30_days|lowest_monthly_earnings|highest_monthly_earnings|lowest_yearly_earnings|highest_yearly_earnings|subscribers_for_last_30_days|created_year|created_month|created_date|Gross_tertiary_education_enrollment|Population|Unemployment_rate|Urban_population|Latitude|Longitude|
+----+--------+-----------+-----------+--------+-----+-------+-------+------------+------------+----------------

In [0]:
duplicates = df.dropDuplicates()

In [0]:
duplicates

Out[26]: DataFrame[rank: string, Youtuber: string, subscribers: string, video_views: string, category: string, Title: string, uploads: string, Country: string, Abbreviation: string, channel_type: string, video_views_rank: string, country_rank: string, channel_type_rank: string, video_views_for_the_last_30_days: string, lowest_monthly_earnings: string, highest_monthly_earnings: string, lowest_yearly_earnings: string, highest_yearly_earnings: string, subscribers_for_last_30_days: string, created_year: string, created_month: string, created_date: string, Gross_tertiary_education_enrollment: string, Population: string, Unemployment_rate: string, Urban_population: string, Latitude: string, Longitude: string]

In [0]:
numeric_columns = ["subscribers", "video_views", "video_views_rank", "country_rank", 
                    "channel_type_rank", "video_views_for_the_last_30_days",
                    "lowest_monthly_earnings", "highest_monthly_earnings", 
                    "lowest_yearly_earnings", "highest_yearly_earnings", 
                    "subscribers_for_last_30_days", "Gross_tertiary_education_enrollment", 
                    "Population", "Unemployment_rate"]

for column in numeric_columns:
    df = df.withColumn(column, col(column).cast(DoubleType()))

# Validate and convert date columns (assuming date columns are in a specific format)
date_columns = ["created_year", "created_month", "created_date"]

for column in date_columns:
    df = df.withColumn(column, col(column).cast(DateType()))

# Validate and convert categorical columns if needed (e.g., converting to StringType)
categorical_columns = ["Youtuber", "category", "Title", "Country", "Abbreviation", "channel_type"]

for column in categorical_columns:
    df = df.withColumn(column, col(column).cast(StringType()))


In [0]:
duplicate_columns = ['Youtuber', 'Country']

df_no_duplicates = df.dropDuplicates(subset=duplicate_columns)


In [0]:
configs = {
  "fs.azure.account.auth.type": "CustomAccessToken",
  "fs.azure.account.custom.token.provider.class": spark.conf.get("spark.databricks.passthrough.adls.gen2.tokenProviderClassName")
}

directory_to_check = "/mnt/gold"
mounted_directories = [mount.mountPoint for mount in dbutils.fs.mounts()]

if directory_to_check in mounted_directories:
    print(f"The directory {directory_to_check} is already mounted.")
else:
    dbutils.fs.mount(
    source = "abfss://gold@youtubeadlsg2.dfs.core.windows.net/",
    mount_point = "/mnt/gold",
    extra_configs = configs)

In [0]:
output_path = '/mnt/gold/data'
df_no_duplicates.write.format('delta').mode('overwrite').save(output_path)